# Build a UI using streamlit for model access using deployed fastapi service

## Introduction to Streamlit for ML Model UI
Streamlit is an open-source Python library that makes it easy to create beautiful web applications for machine learning and data science projects. This lab demonstrates how to build an interactive user interface that connects to our deployed FastAPI service, allowing users to make predictions through a user-friendly web interface.

## Why Use Streamlit with FastAPI?
Streamlit combined with FastAPI creates a complete end-to-end ML application by providing:

- User-Friendly Interface: Interactive forms and widgets for data input
- Real-time Predictions: Seamless integration with FastAPI backend
- Rapid Prototyping: Quick development of ML application frontends
- No Frontend Knowledge Required: Build web apps using only Python

This notebook builds a Streamlit frontend for our churn prediction FastAPI service, creating a complete ML application stack.

## Setup
### First, let's install Streamlit and connect to our existing FastAPI service.

In [1]:
# Install Streamlit
!pip install streamlit pyngrok requests -q

## Import Libraries

In [2]:
import pandas as pd
import streamlit as st
import requests
import json
import subprocess
import threading
import time
from pyngrok import ngrok
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import uvicorn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Create FastAPI Service
### First, we'll create a standalone FastAPI service with a trained model.

In [4]:
# Load and preprocess data
df = pd.read_csv("/content/drive/MyDrive/cognixia_labs_test/Week_2_labs/Lab_3 Streamlit UI for model and fastapi service/telecome data.csv")
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

feature_columns = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
                   'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
                   'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
                   'MonthlyCharges', 'TotalCharges']
X = df[feature_columns].copy()
y = df['Churn'].copy()

# Encode categorical variables
label_encoders = {}
categorical_columns = X.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Encode target variable
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(y)

# Split and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print(f"Model trained with accuracy: {model.score(X_test, y_test):.3f}")

# Create FastAPI app
app = FastAPI()

class PredictionInput(BaseModel):
    data: List[List[float]]

@app.post("/predict")
async def predict(input_data: PredictionInput):
    try:
        input_df = pd.DataFrame(input_data.data, columns=feature_columns)
        for col in categorical_columns:
            input_df[col] = input_df[col].astype(int)
        predictions = model.predict(input_df)
        predictions = target_encoder.inverse_transform(predictions)
        return {"predictions": predictions.tolist()}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# Start FastAPI server
def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=5000)

threading.Thread(target=run_fastapi, daemon=True).start()
time.sleep(5)
print("FastAPI service started successfully!")

/tmp/ipython-input-4-2978526196.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
INFO:     Started server process [10437]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


Model trained with accuracy: 0.796
FastAPI service started successfully!


## Explanation:

- Data Processing: Load and preprocess the telecom dataset
- Model Training: Train a Random Forest classifier for churn prediction
- FastAPI Service: Create REST API endpoint for predictions
- Background Server: Run FastAPI in a separate thread

## Streamlit UI Application
### Create an interactive Streamlit application that connects to the FastAPI service.

In [5]:
# Create simple Streamlit app file
streamlit_app_code = '''
import streamlit as st
import requests

# Title
st.title("Churn Prediction App")

# FastAPI endpoint URL
API_URL = "FASTAPI_URL_PLACEHOLDER"

# Input form
st.header("Customer Information")

# Simple input fields
gender = st.selectbox("Gender", ["Female", "Male"])
senior_citizen = st.selectbox("Senior Citizen", ["No", "Yes"])
partner = st.selectbox("Partner", ["No", "Yes"])
dependents = st.selectbox("Dependents", ["No", "Yes"])
tenure = st.number_input("Tenure (months)", 0, 72, 1)
phone_service = st.selectbox("Phone Service", ["No", "Yes"])
multiple_lines = st.selectbox("Multiple Lines", ["No", "Yes", "No phone service"])
internet_service = st.selectbox("Internet Service", ["DSL", "Fiber optic", "No"])
online_security = st.selectbox("Online Security", ["No", "Yes", "No internet service"])
online_backup = st.selectbox("Online Backup", ["No", "Yes", "No internet service"])
device_protection = st.selectbox("Device Protection", ["No", "Yes", "No internet service"])
tech_support = st.selectbox("Tech Support", ["No", "Yes", "No internet service"])
streaming_tv = st.selectbox("Streaming TV", ["No", "Yes", "No internet service"])
streaming_movies = st.selectbox("Streaming Movies", ["No", "Yes", "No internet service"])
contract = st.selectbox("Contract", ["Month-to-month", "One year", "Two year"])
paperless_billing = st.selectbox("Paperless Billing", ["No", "Yes"])
payment_method = st.selectbox("Payment Method", ["Electronic check", "Mailed check", "Bank transfer (automatic)", "Credit card (automatic)"])
monthly_charges = st.number_input("Monthly Charges ($)", 18.0, 120.0, 50.0)
total_charges = st.number_input("Total Charges ($)", 18.0, 8685.0, 1000.0)

# Predict button
if st.button("Predict Churn"):
    # Encode categorical variables
    def encode_categorical(value, options):
        return options.index(value)

    # Prepare input data
    input_data = [
        1 if gender == "Male" else 0,
        1 if senior_citizen == "Yes" else 0,
        1 if partner == "Yes" else 0,
        1 if dependents == "Yes" else 0,
        tenure,
        1 if phone_service == "Yes" else 0,
        encode_categorical(multiple_lines, ["No", "No phone service", "Yes"]),
        encode_categorical(internet_service, ["DSL", "Fiber optic", "No"]),
        encode_categorical(online_security, ["No", "No internet service", "Yes"]),
        encode_categorical(online_backup, ["No", "No internet service", "Yes"]),
        encode_categorical(device_protection, ["No", "No internet service", "Yes"]),
        encode_categorical(tech_support, ["No", "No internet service", "Yes"]),
        encode_categorical(streaming_tv, ["No", "No internet service", "Yes"]),
        encode_categorical(streaming_movies, ["No", "No internet service", "Yes"]),
        encode_categorical(contract, ["Month-to-month", "One year", "Two year"]),
        1 if paperless_billing == "Yes" else 0,
        encode_categorical(payment_method, ["Bank transfer (automatic)", "Credit card (automatic)", "Electronic check", "Mailed check"]),
        monthly_charges,
        total_charges
    ]

    # Make API request
    try:
        response = requests.post(f"{API_URL}/predict", json={"data": [input_data]})
        if response.status_code == 200:
            prediction = response.json()["predictions"][0]
            if prediction == "Yes":
                st.error("Customer will likely CHURN")
            else:
                st.success("Customer will likely STAY")
        else:
            st.error("Prediction failed")
    except:
        st.error("Cannot connect to API")
'''

# Write Streamlit app to file
with open('/content/streamlit_app.py', 'w') as f:
    f.write(streamlit_app_code)

print("Streamlit app created successfully!")

Streamlit app created successfully!


## Explanation:

- Page Configuration: Sets up the Streamlit page with title, icon, and layout
- Interactive Widgets: Uses selectboxes, sliders, and number inputs for user data collection
- Data Encoding: Converts categorical inputs to numerical format matching the model training
- API Integration: Sends POST requests to the FastAPI prediction endpoint
- Results Display: Shows predictions with color-coded styling and risk factor analysis

## Deploy FastAPI Service
### Deploy the FastAPI service with ngrok to get a public URL.

In [6]:
# Expose FastAPI with ngrok
ngrok_token = "2yGBZnB7ngE0P19IGz2Qf2DW2EW_3Bj4VstSQQiy16UdWqKvX"
ngrok.set_auth_token(ngrok_token)
ngrok.kill()

# Deploy FastAPI
fastapi_url = ngrok.connect(5000, bind_tls=True).public_url
print(f"FastAPI service deployed at: {fastapi_url}")

# Test FastAPI endpoint
sample_data = {"data": [[1, 0, 1, 0, 34, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 56.95, 1889.5]]}
response = requests.post(f"{fastapi_url}/predict", json=sample_data)
print(f"FastAPI test response: {response.json()}")

FastAPI service deployed at: https://46c96c357f64.ngrok-free.app
INFO:     34.126.96.123:0 - "POST /predict HTTP/1.1" 200 OK
FastAPI test response: {'predictions': ['No']}


## Explanation:

- Replace FASTAPI_URL with the actual URL from your deployed FastAPI service
- The app dynamically updates to connect to your specific FastAPI endpoint

## Create and Deploy Streamlit Application
### Now create and deploy the Streamlit UI that users will interact with.

In [7]:
# Update Streamlit app with FastAPI URL
with open('/content/streamlit_app.py', 'r') as f:
    streamlit_code = f.read()

streamlit_code = streamlit_code.replace('FASTAPI_URL_PLACEHOLDER', fastapi_url)

with open('/content/streamlit_app.py', 'w') as f:
    f.write(streamlit_code)

print(f"Streamlit app configured with FastAPI URL: {fastapi_url}")

# Start Streamlit server
def run_streamlit():
    subprocess.run(['streamlit', 'run', '/content/streamlit_app.py',
                   '--server.port', '8501', '--server.address', '0.0.0.0'])

threading.Thread(target=run_streamlit, daemon=True).start()
time.sleep(10)

# Deploy Streamlit UI with ngrok
streamlit_url = ngrok.connect(8501, bind_tls=True).public_url
print(f"🎯 Streamlit UI is live at: {streamlit_url}")

Streamlit app configured with FastAPI URL: https://46c96c357f64.ngrok-free.app
🎯 Streamlit UI is live at: https://4059ce81ef79.ngrok-free.app


## Explanation:

- URL Configuration: Updates the Streamlit app with the actual FastAPI URL
- Streamlit Server: Starts the Streamlit server on port 8501
- Background Process: Runs Streamlit in a separate thread
- Public Access: Creates a secure public URL for the Streamlit interface



## Test the Application
### Verify that the Streamlit UI can successfully connect to the FastAPI service.

In [8]:
# Test connection to FastAPI
try:
    test_response = requests.get(f"{fastapi_url}/docs")
    print("✅ FastAPI service is accessible")
except:
    print("⚠️ FastAPI service connection issue")

print("\\n" + "="*50)
print("🚀 COMPLETE APPLICATION DEPLOYED!")
print("="*50)
print(f"🎯 Streamlit UI:    {streamlit_url}")
print("="*50)

INFO:     34.126.96.123:0 - "GET /docs HTTP/1.1" 200 OK
✅ FastAPI service is accessible
\n==================================================
🚀 COMPLETE APPLICATION DEPLOYED!
🎯 Streamlit UI:    https://4059ce81ef79.ngrok-free.app
